# 04 - Advanced Pipelines

This notebook covers advanced topics:

1. Creating Custom Pipelines
2. Pipeline Optimization (Quantization, ONNX)
3. Using Local Models
4. Pipeline Callbacks and Hooks
5. Combining Multiple Pipelines
6. Error Handling and Edge Cases

In [ ]:
from transformers import (
    pipeline, 
    Pipeline,
    AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig
)
import torch
import numpy as np

---
## 1. Creating Custom Pipelines

Sometimes you need functionality not covered by built-in pipelines.

### 1.1 Custom Pipeline from Scratch

In [ ]:
from transformers import Pipeline

class SimilarityPipeline(Pipeline):
    """
    Custom pipeline to compute text similarity scores.
    """
    
    def _sanitize_parameters(self, **kwargs):
        """Handle pipeline parameters."""
        preprocess_kwargs = {}
        postprocess_kwargs = {}
        
        if "text_pair" in kwargs:
            preprocess_kwargs["text_pair"] = kwargs["text_pair"]
        
        return preprocess_kwargs, {}, postprocess_kwargs
    
    def preprocess(self, text, text_pair=None):
        """Tokenize the input texts."""
        if text_pair is None:
            raise ValueError("text_pair is required for similarity")
        
        # Tokenize both texts
        inputs = self.tokenizer(
            text, 
            text_pair,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        return inputs
    
    def _forward(self, model_inputs):
        """Run the model."""
        outputs = self.model(**model_inputs)
        return outputs
    
    def postprocess(self, model_outputs):
        """Process model outputs into similarity scores."""
        # Get the [CLS] token embedding
        logits = model_outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        
        # Assume binary classification (similar/not similar)
        similarity_score = probabilities[0][1].item()
        
        return {
            "similarity": similarity_score,
            "label": "similar" if similarity_score > 0.5 else "not similar"
        }

print("Custom SimilarityPipeline class defined!")

In [ ]:
# Using the custom pipeline
# Note: This requires an appropriate model trained for similarity

# For demonstration, we'll use a paraphrase model
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "textattack/bert-base-uncased-MRPC"  # Paraphrase detection model
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create custom pipeline
similarity_pipe = SimilarityPipeline(
    model=model,
    tokenizer=tokenizer,
    device=-1  # CPU
)

# Test it
result = similarity_pipe(
    "The cat sat on the mat.",
    text_pair="A cat was sitting on a mat."
)
print(f"Result: {result}")

### 1.2 Extending Existing Pipelines

In [ ]:
from transformers import TextClassificationPipeline

class EnhancedClassificationPipeline(TextClassificationPipeline):
    """
    Extended classification pipeline with additional features.
    """
    
    def postprocess(self, model_outputs, function_to_apply=None, top_k=1, _legacy=True):
        # Call parent's postprocess
        results = super().postprocess(
            model_outputs, 
            function_to_apply=function_to_apply,
            top_k=top_k,
            _legacy=_legacy
        )
        
        # Add custom fields
        for result in results:
            # Add confidence level
            score = result['score']
            if score > 0.9:
                result['confidence'] = 'high'
            elif score > 0.7:
                result['confidence'] = 'medium'
            else:
                result['confidence'] = 'low'
            
            # Add emoji based on sentiment
            if 'POSITIVE' in result['label'].upper():
                result['emoji'] = '😊'
            elif 'NEGATIVE' in result['label'].upper():
                result['emoji'] = '😞'
            else:
                result['emoji'] = '😐'
        
        return results

# Use the enhanced pipeline
enhanced_classifier = EnhancedClassificationPipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

result = enhanced_classifier("I absolutely love this!")
print(f"Enhanced result: {result}")

---
## 2. Pipeline Optimization

### 2.1 Using Half Precision (FP16)

In [ ]:
# FP16 reduces memory by ~50% with minimal accuracy loss
if torch.cuda.is_available():
    classifier_fp16 = pipeline(
        "sentiment-analysis",
        model="distilbert-base-uncased-finetuned-sst-2-english",
        torch_dtype=torch.float16,
        device=0
    )
    print("FP16 pipeline created on GPU")
else:
    print("GPU not available - FP16 works best on GPU")

### 2.2 Model Quantization (INT8)

In [ ]:
# Dynamic quantization for CPU inference
# This reduces model size and speeds up CPU inference

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},  # Quantize linear layers
    dtype=torch.qint8
)

# Create pipeline with quantized model
quantized_classifier = pipeline(
    "sentiment-analysis",
    model=quantized_model,
    tokenizer=tokenizer
)

result = quantized_classifier("This is great!")
print(f"Quantized model result: {result}")

### 2.3 Using ONNX Runtime

In [ ]:
# ONNX provides optimized inference
# Install: pip install optimum[onnxruntime]

try:
    from optimum.onnxruntime import ORTModelForSequenceClassification
    from transformers import AutoTokenizer, pipeline
    
    # Load model in ONNX format
    model = ORTModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english",
        export=True  # Convert to ONNX
    )
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
    
    # Create ONNX-optimized pipeline
    onnx_classifier = pipeline(
        "sentiment-analysis",
        model=model,
        tokenizer=tokenizer
    )
    
    result = onnx_classifier("ONNX is fast!")
    print(f"ONNX result: {result}")
    
except ImportError:
    print("Install optimum: pip install optimum[onnxruntime]")

### 2.4 Batching for Performance

In [ ]:
import time

classifier = pipeline("sentiment-analysis")

# Generate test data
texts = [f"This is test sentence number {i}." for i in range(100)]

# Method 1: One at a time (slow)
start = time.time()
results_slow = [classifier(t) for t in texts]
time_slow = time.time() - start
print(f"One at a time: {time_slow:.2f}s")

# Method 2: Batch processing (faster)
start = time.time()
results_batch = classifier(texts, batch_size=16)
time_batch = time.time() - start
print(f"Batch (size=16): {time_batch:.2f}s")

print(f"Speedup: {time_slow/time_batch:.1f}x")

---
## 3. Using Local Models

### 3.1 Save and Load Models Locally

In [ ]:
import os

# Load model from Hub
classifier = pipeline("sentiment-analysis")

# Save locally
save_dir = "./local_model"
classifier.model.save_pretrained(save_dir)
classifier.tokenizer.save_pretrained(save_dir)

print(f"Model saved to {save_dir}")
print(f"Files: {os.listdir(save_dir)}")

In [ ]:
# Load from local directory
local_classifier = pipeline(
    "sentiment-analysis",
    model=save_dir,
    tokenizer=save_dir
)

result = local_classifier("Testing local model!")
print(f"Local model result: {result}")

### 3.2 Offline Mode

In [ ]:
# For air-gapped environments
import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["HF_DATASETS_OFFLINE"] = "1"

# Now the library won't try to download anything
# It will only use locally cached models

# Reset for this notebook
os.environ["TRANSFORMERS_OFFLINE"] = "0"
print("Offline mode disabled for this notebook")

---
## 4. Combining Multiple Pipelines

### 4.1 Sequential Pipeline Chain

In [ ]:
class PipelineChain:
    """
    Chain multiple pipelines together.
    """
    def __init__(self):
        self.ner = pipeline("ner", grouped_entities=True)
        self.sentiment = pipeline("sentiment-analysis")
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    
    def analyze(self, text):
        """Run comprehensive text analysis."""
        results = {
            'original_text': text,
            'char_count': len(text),
            'word_count': len(text.split())
        }
        
        # 1. Extract entities
        entities = self.ner(text)
        results['entities'] = [
            {'text': e['word'], 'type': e['entity_group'], 'score': round(e['score'], 3)}
            for e in entities
        ]
        
        # 2. Sentiment
        sentiment = self.sentiment(text)[0]
        results['sentiment'] = {
            'label': sentiment['label'],
            'score': round(sentiment['score'], 3)
        }
        
        # 3. Summary (if text is long enough)
        if len(text.split()) > 50:
            summary = self.summarizer(text, max_length=50, min_length=20)
            results['summary'] = summary[0]['summary_text']
        else:
            results['summary'] = 'Text too short for summarization'
        
        return results

# Usage
chain = PipelineChain()

text = """
Apple Inc. reported record quarterly revenue of $123.9 billion today, 
driven by strong iPhone sales. CEO Tim Cook announced the results from 
their headquarters in Cupertino, California. The company's stock rose 
5% in after-hours trading. Analysts from Goldman Sachs and Morgan Stanley 
praised the results, calling it a "exceptional performance" despite 
global supply chain challenges.
"""

analysis = chain.analyze(text)
print("=" * 50)
print("TEXT ANALYSIS REPORT")
print("=" * 50)
print(f"\nWord Count: {analysis['word_count']}")
print(f"\nSentiment: {analysis['sentiment']['label']} ({analysis['sentiment']['score']})")
print(f"\nEntities Found:")
for e in analysis['entities']:
    print(f"  - {e['type']}: {e['text']}")
print(f"\nSummary: {analysis['summary']}")

### 4.2 Parallel Pipeline Execution

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

class ParallelPipelines:
    """
    Run multiple pipelines in parallel.
    """
    def __init__(self):
        self.pipelines = {
            'sentiment': pipeline("sentiment-analysis"),
            'ner': pipeline("ner", grouped_entities=True),
            'zero_shot': pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        }
    
    def _run_sentiment(self, text):
        return {'sentiment': self.pipelines['sentiment'](text)}
    
    def _run_ner(self, text):
        return {'ner': self.pipelines['ner'](text)}
    
    def _run_zero_shot(self, text, labels):
        return {'topics': self.pipelines['zero_shot'](text, labels)}
    
    def analyze_parallel(self, text, topic_labels):
        """Run all analyses in parallel."""
        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = [
                executor.submit(self._run_sentiment, text),
                executor.submit(self._run_ner, text),
                executor.submit(self._run_zero_shot, text, topic_labels)
            ]
            
            results = {}
            for future in futures:
                results.update(future.result())
        
        return results

# Usage
parallel = ParallelPipelines()

text = "Microsoft announced a new partnership with OpenAI to develop advanced AI systems."
labels = ["technology", "business", "science", "sports"]

start = time.time()
results = parallel.analyze_parallel(text, labels)
elapsed = time.time() - start

print(f"Parallel analysis completed in {elapsed:.2f}s")
print(f"\nSentiment: {results['sentiment']}")
print(f"Entities: {[e['word'] for e in results['ner']]}")
print(f"Top topic: {results['topics']['labels'][0]}")

---
## 5. Error Handling and Edge Cases

In [ ]:
class RobustPipeline:
    """
    Pipeline wrapper with error handling.
    """
    def __init__(self, task, model=None, max_length=512):
        self.pipe = pipeline(task, model=model)
        self.max_length = max_length
        self.task = task
    
    def __call__(self, text, **kwargs):
        # Handle empty input
        if not text or not text.strip():
            return {'error': 'Empty input', 'result': None}
        
        # Handle very long text
        if len(text) > self.max_length * 4:  # Rough char estimate
            text = text[:self.max_length * 4]
            truncated = True
        else:
            truncated = False
        
        try:
            result = self.pipe(text, **kwargs)
            return {
                'result': result,
                'truncated': truncated,
                'error': None
            }
        except Exception as e:
            return {
                'error': str(e),
                'result': None,
                'truncated': truncated
            }

# Usage
robust = RobustPipeline("sentiment-analysis")

# Test with various inputs
test_cases = [
    "Normal text that works fine.",
    "",  # Empty
    "   ",  # Whitespace only
    "A " * 10000  # Very long
]

for i, text in enumerate(test_cases):
    result = robust(text)
    print(f"Test {i+1}: error={result['error']}, truncated={result.get('truncated', False)}")

---
## 🎯 Key Takeaways

1. **Custom Pipelines**: Inherit from `Pipeline` or existing pipeline classes
2. **Optimization**: FP16, quantization, and ONNX can significantly improve performance
3. **Batching**: Always batch when processing multiple inputs
4. **Local Models**: Save/load for offline or production use
5. **Chaining**: Combine pipelines for complex workflows
6. **Error Handling**: Always wrap pipelines for production use

## Next Steps

Continue to [05_production_patterns.ipynb](05_production_patterns.ipynb) for production best practices!